In [3]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("../Nemotron_Personas.parquet")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   uuid                        100000 non-null  object
 1   persona                     100000 non-null  object
 2   professional_persona        100000 non-null  object
 3   sports_persona              100000 non-null  object
 4   arts_persona                100000 non-null  object
 5   travel_persona              100000 non-null  object
 6   culinary_persona            100000 non-null  object
 7   skills_and_expertise        100000 non-null  object
 8   skills_and_expertise_list   100000 non-null  object
 9   hobbies_and_interests       100000 non-null  object
 10  hobbies_and_interests_list  100000 non-null  object
 11  career_goals_and_ambitions  100000 non-null  object
 12  sex                         100000 non-null  object
 13  age                         10

In [5]:
for column in df.columns:
    value=df.loc[1,column]
    print(f"Column: {column} \n Value: \n{value}\n")

Column: uuid 
 Value: 
3b5691bf-07cd-4e58-b85b-cff62faba2fd

Column: persona 
 Value: 
Quintin, a 40-year-old logistician from Converse, TX, balances curiosity with practicality, appreciating both new ideas and established methods, and maintains a unique blend of organization and flexibility in all aspects of life.

Column: professional_persona 
 Value: 
Quintin Pete Johnson, a logistician, combines his practical nature with a curiosity for new technologies, constantly seeking to optimize supply chains while remaining grounded in proven methods.

Column: sports_persona 
 Value: 
Quintin Pete Johnson, a dedicated fan of the San Antonio Spurs, maintains a balanced fitness routine, playing pick-up basketball on weekends and hiking at Government Canyon State Natural Area during weekdays.

Column: arts_persona 
 Value: 
They appreciate the gritty realism of Texas artist David Adickes' sculptures and the soulful melodies of Tejano musician Selena, often visiting the San Antonio Museum of Art

In [29]:
df.columns

Index(['uuid', 'persona', 'professional_persona', 'sports_persona',
       'arts_persona', 'travel_persona', 'culinary_persona',
       'skills_and_expertise', 'skills_and_expertise_list',
       'hobbies_and_interests', 'hobbies_and_interests_list',
       'career_goals_and_ambitions', 'sex', 'age', 'marital_status',
       'education_level', 'bachelors_field', 'occupation', 'city', 'state',
       'zipcode', 'country'],
      dtype='object')

In [ ]:
not_required_columns=['zipcode', 'country']
persona_columns=['persona', 'professional_persona', 'sports_persona',
                'arts_persona', 'travel_persona', 'culinary_persona']
categorical_columns=['sex', 'marital_status','education_level', 
                     'bachelors_field', 'occupation', 'city', 'state']

In [9]:
df[persona_columns].head()

,persona,professional_persona,sports_persona,arts_persona,travel_persona,culinary_persona
0,"A disciplined, sociable visionary, Jonathan ba...","A retired manufacturing manager, Jonathan now ...","An avid golfer, Jonathan plays weekly at the W...","A history enthusiast, Jonathan often leads tou...","A seasoned, meticulous planner, Jonathan favor...","A fan of hearty, Midwestern comfort food, Jona..."
1,"Quintin, a 40-year-old logistician from Conver...","Quintin Pete Johnson, a logistician, combines ...","Quintin Pete Johnson, a dedicated fan of the S...",They appreciate the gritty realism of Texas ar...,"Quintin, a meticulous planner, balances family...",They delight in preparing complex Tex-Mex dish...
2,"Ashley, a passionate community advocate, balan...","Ashley, an aspiring union representative, exce...","Ashley, a dedicated Detroit Lions fan, maintai...","Ashley, a self-proclaimed 'Motown music enthus...","Ashley, a budget-conscious traveler, dreams of...","Ashley, a skilled home cook, loves preparing h..."
3,"Stephanie, always the first to volunteer, bala...","Stephanie, a customer service representative, ...","Stephanie, a die-hard Minnesota Vikings fan, p...","Stephanie, an avid reader and amateur painter,...","Stephanie, despite her love for the outdoors, ...","Stephanie, a self-taught cook, enjoys experime..."
4,"Sonia, a 70-year-old retiree, is a vibrant, im...","Sonia, a retired organizer with a creative sou...","Sonia, though not athletic, enjoys watching ba...","Sonia, a passionate artist, finds inspiration ...","Sonia, a seasoned traveler, plans meticulous i...","Sonia, an avid cook, delights in preparing com..."


In [11]:
df[categorical_columns].nunique()

sex                   2
marital_status        5
education_level       7
bachelors_field       5
occupation          566
city               9634
state                52
dtype: int64

## expertise_list

-TF-IDF 

In [16]:
import ast
df['hobbies_and_interests_list'] = df['hobbies_and_interests_list'].apply(ast.literal_eval)

In [26]:
total_hobbies = set()
for hobbies in df['hobbies_and_interests_list']:
    total_hobbies.update(hobbies)


len(total_hobbies)

42615

In [27]:
total_hobbies

{'fishing and camping with a touch of comfort',
 'model airplane flying',
 'volunteering (coding workshops)',
 'obscure topics lectures',
 'reading, particularly classic literature and science fiction',
 'exploring las vegas nightlife',
 'experimenting with new painting techniques',
 'playing football (soccer)',
 'people-watching in wynwood',
 'watching independent and foreign films',
 'listening to traditional irish music',
 'art enthusiast',
 'librarianship',
 'reading art books and journals',
 'following louisville cardinals',
 'painting and pottery, with a focus on learning traditional native american techniques',
 'coaching ffa team',
 'cooking soul food and trying new recipes',
 'attending jazz clubs and art exhibitions',
 'reading, especially history and biographies',
 'art program volunteer',
 'small concerts and open mic events',
 'cooking traditional black american dishes',
 'reading (cozy mysteries, historical romances)',
 'exploring new places and cultures',
 'experimental 

In [14]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

df_combined=pd.DataFrame()

# # Combine the text columns into a single 'full_persona' column
text_columns = persona_columns+['skills_and_expertise', 'skills_and_expertise_list', 'hobbies_and_interests', 'hobbies_and_interests_list', 'career_goals_and_ambitions']
df_combined['full_persona'] = df[text_columns].apply(lambda row: ' \n'.join(row.values.astype(str)), axis=1)


In [ ]:

# # Create a single vector for the combined text
model = SentenceTransformer('Qwen/Qwen3-Embedding-0.6B')
test_sentence = model.encode(df_combined['full_persona'][0:1].tolist())
# df_combined['persona_vector'] = df_combined['full_persona'].apply(lambda text: model.encode(text))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\alt_user\anaconda3\envs\prop_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alt_user\.cache\huggingface\hub\models--Qwen--Qwen3-Embedding-0.6B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [ ]:
test_sentence

array([[ 0.03068872,  0.00593236, -0.01115962, ..., -0.02292692,
        -0.02908194, -0.02015192]], dtype=float32)

In [ ]:
df_combined['full_persona'][0:1].tolist()

["A disciplined, sociable visionary, Jonathan balances practicality with curiosity, leaving a lasting impact on his community through his organized, competitive approach \nA retired manufacturing manager, Jonathan now excels as a community developer, leveraging his organizational skills and competitive nature to drive sustainable growth in Wickliffe \nAn avid golfer, Jonathan plays weekly at the Wickliffe Country Club and cheers for the Cleveland Browns, maintaining his competitive spirit even in leisure \nA history enthusiast, Jonathan often leads tours at the Lake County Historical Society, sharing stories about local pioneers and their impact on the region's development \nA seasoned, meticulous planner, Jonathan favors international destinations with rich histories, like Edinburgh and Dublin, where he can explore ancestral roots and enjoy a round of golf at prestigious courses \nA fan of hearty, Midwestern comfort food, Jonathan enjoys cooking traditional family recipes, like his gr